In [1]:
import sys
import torch
import torch.nn as nn
from torch.autograd import Variable
import numpy as np

In [20]:
torch.manual_seed(777) #reproducibility

idx2char = ['h','i','e','l','o']

# # Teach hihell -> ihello
# x_data = [0, 1, 0, 2, 3, 3]   # hihell
# one_hot_lookup = np.identity(len(idx2char))
# one_hot_lookup
# Teach hihell -> ihello
x_data = [0, 1, 0, 2, 3, 3]   # hihell
one_hot_lookup = [[1, 0, 0, 0, 0],  # 0
                  [0, 1, 0, 0, 0],  # 1
                  [0, 0, 1, 0, 0],  # 2
                  [0, 0, 0, 1, 0],  # 3
                  [0, 0, 0, 0, 1]]  # 4
one_hot_lookup

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 0, 1]]

In [21]:
y_data=[1,0,2,3,3,4]
x_one_hot = [one_hot_lookup[x] for x in x_data]
x_one_hot

[[1, 0, 0, 0, 0],
 [0, 1, 0, 0, 0],
 [1, 0, 0, 0, 0],
 [0, 0, 1, 0, 0],
 [0, 0, 0, 1, 0],
 [0, 0, 0, 1, 0]]

In [22]:
#make a tensor
inputs = Variable(torch.Tensor(x_one_hot))
outputs = Variable(torch.LongTensor(y_data))
print(inputs)
print(outputs)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 1., 0.]])
tensor([1, 0, 2, 3, 3, 4])


In [27]:
num_classes = 5
input_size = len(one_hot_lookup[0])
hidden_size = 5
batch_size = 1
sequence_length = 1
num_layers = 1

# class Model(nn.Module):
#     def __init__(self):
#         super(Model,self).__init__()
#         self.rnn = nn.RNN(input_size=input_size,
#                          hidden_size=hidden_size, batch_first=True)
        
#     def forward(self,hidden,x):
#         x = x.view(batch_size,sequence_length,input_size)
#         print('x: ',x)
#         out,hidden = self.rnn(x,hidden)
#         print('out: ',out)
#         print('hidden: ',hidden)
#         return hidden, out.view(-1,num_classes)
    
#     def init_hidden(self):
        
#         return Variable(torch.zeros(num_layer,batch_size,hidden_size))
    
# #inisialisasi
# model = Model()
# print(model)

class Model(nn.Module):

    def __init__(self):
        super(Model, self).__init__()
        self.rnn = nn.RNN(input_size=input_size,
                          hidden_size=hidden_size, batch_first=True)

    def forward(self, hidden, x):
        # Reshape input (batch first)
        x = x.view(batch_size, sequence_length, input_size)

        # Propagate input through RNN
        # Input: (batch, seq_len, input_size)
        # hidden: (num_layers * num_directions, batch, hidden_size)
        out, hidden = self.rnn(x, hidden)
        return hidden, out.view(-1, num_classes)

    def init_hidden(self):
        # Initialize hidden and cell states
        # (num_layers * num_directions, batch, hidden_size)
        return Variable(torch.zeros(num_layers, batch_size, hidden_size))


# Instantiate RNN model
model = Model()
print(model)


Model(
  (rnn): RNN(5, 5, batch_first=True)
)


In [31]:
# criterion = nn.CrossEntropyLoss()
# optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# #train model
# for epoch in range(10):
#     optimizer.zero_grad()
#     loss = 0
#     hidden = model.init_hidden()
    
#     sys.stdout.write("predicted string: ")
#     for input_, label in zip(inputs,outputs):
#         hidden,output = model(hidden,input_)
#         print('hidden: ',hidden)
#         print('output: ', output)
#         val, idx = output.max(1)
#         print('val: ',val)
#         print('idx: ',idx)
#         sys.stdout.write(idx2char[idx.data[0]])
#         print('label ',label)
#         print(output)
#         loss += criterion(output,label)
        
#     print('epoch %d, loss: %1.3f' % (epoch+1, loss.item()))
    
#     loss.backward()
#     optimizer.step()

# Set loss and optimizer function
# CrossEntropyLoss = LogSoftmax + NLLLoss
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.1)

# Train the model
for epoch in range(100):
    optimizer.zero_grad()
    loss = 0
    hidden = model.init_hidden()

    sys.stdout.write("predicted string: ")
    for input, label in zip(inputs, outputs):
        # print(input.size(), label.size())
        hidden, output = model(hidden, input)
        val, idx = output.max(1)
        sys.stdout.write(idx2char[idx.item()])
        loss += criterion(output, label)

    print(", epoch: %d, loss: %1.3f" % (epoch + 1, loss.item()))

    loss.backward()
    optimizer.step()


    
print("learning finish")

predicted string: l

RuntimeError: dimension specified as 0 but tensor has no dimensions